In [0]:
import os
from xml.dom import minidom
import pandas as pd
import numpy as np
import nltk
import gzip
import gensim 
import logging,re
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Extracting text from Query and Case documents**

In [0]:
directory = os.getcwd() 
dirr = '/content/drive/My Drive/AILA-data/'
tmp = open(dirr+"Query_doc.txt", "r") 
temp = tmp.read()
tmp.close()
queries = temp.split("\n")
queries.remove(queries[-1])
query_docs = []
query_labels = []
query_words = []
for i in range(len(queries)):
    elem = queries[i].split("||")
    query_docs.append(elem[1])
    query_labels.append(elem[0])
    query_words.append(re.split('\W+',elem[1]))

In [0]:
# Extract text from inside

def preprocess(directory):
    case_files = os.listdir(directory)
    case_docs =[]
    case_words = []
    for i in range(len(case_files)):
        if i%50 ==0:
            print("Processing : " , i+1 )
        path = directory+'/'+case_files[i]
        f = open(path, "r")
        case_docs.append(f.read())
        case_words.append(re.split('\W+',case_docs[i]))
    return case_docs,case_words,case_files

prior_case_docs, prior_case_words,prior_labels = preprocess(dirr+'Object_casedocs')
    
data =  query_docs + prior_case_docs
words = query_words + prior_case_words
labels = query_labels + prior_labels
    


Processing :  1
Processing :  51
Processing :  101
Processing :  151
Processing :  201
Processing :  251
Processing :  301
Processing :  351
Processing :  401
Processing :  451
Processing :  501
Processing :  551
Processing :  601
Processing :  651
Processing :  701
Processing :  751
Processing :  801
Processing :  851
Processing :  901
Processing :  951
Processing :  1001
Processing :  1051
Processing :  1101
Processing :  1151
Processing :  1201
Processing :  1251
Processing :  1301
Processing :  1351
Processing :  1401
Processing :  1451
Processing :  1501
Processing :  1551
Processing :  1601
Processing :  1651
Processing :  1701
Processing :  1751
Processing :  1801
Processing :  1851
Processing :  1901
Processing :  1951
Processing :  2001
Processing :  2051
Processing :  2101
Processing :  2151
Processing :  2201
Processing :  2251
Processing :  2301
Processing :  2351
Processing :  2401
Processing :  2451
Processing :  2501
Processing :  2551
Processing :  2601
Processing :  26

**Pre-processing the text**

In [0]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')
lemma = WordNetLemmatizer()
lemma_dictionary = {}
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())
def filt(case_docs):
    filtered_words =[]
    filtered_docs = [ '' for i in range(len(case_docs))]
    lemma_docs = [ '' for i in range(len(case_docs))]
    for i in range(len(case_docs)):
        if i%50 ==0:
            print("Processing : ", i)
        case_words = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", case_docs[i])
        filtered_word_list = [word.lower() for word in case_words if (( len(word) >= 3 and word.lower() not in stop_words ))  ]  #removing stop words
        filtered_words.append(filtered_word_list)
        for word in filtered_word_list:
            filtered_docs[i] = filtered_docs[i] + word + " "  
            lemma_docs[i] = lemma_docs[i] + lemma.lemmatize(word) + " "     #word net lemmatization
    return lemma_docs
filtered_docs = filt(data)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
Processing :  0
Processing :  50
Processing :  100
Processing :  150
Processing :  200
Processing :  250
Processing :  300
Processing :  350
Processing :  400
Processing :  450
Processing :  500
Processing :  550
Processing :  600
Processing :  650
Processing :  700
Processing :  750
Processing :  800
Processing :  850
Processing :  900
Processing :  950
Processing :  1000
Processing :  1050
Processing :  1100
Processing :  1150
Processing :  1200
Processing :  1250
Processing :  1300
Processing :  1350
Processing :  1400
Processing :  1450
Processing :  1500
Processing :  1550
Processing :  1600
Processing :  1650
Processing :  1700
Processing :  1750
Processing :  1800

**Using Doc2Vec Model of Vector Dimension 150 and epochs 50 and window size 20**

In [0]:
import os
from os import listdir
from os.path import isfile, join
from gensim import models
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Word2Vec
from gensim import corpora, models, similarities
import gensim, logging
import time
import re
import nltk
import io
from nltk.corpus import stopwords
import numpy as np

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())



class DocIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list

    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=re.split('\W+',doc), tags=[self.labels_list[idx]])
iterator = DocIterator(filtered_docs, labels)

model = gensim.models.Doc2Vec(vector_size=150, window=20, min_count=2, workers=4, alpha=0.025, min_alpha=0.001)
model.build_vocab(iterator)

print('done building vocabulary')
print('start training the model')
tic = time.time()
model.train(iterator,total_examples=model.corpus_count, epochs = 50)
toc = time.time()
print(toc-tic)
model.save("doc2vec_case.model")

2020-05-03 13:24:15,408 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-05-03 13:24:15,424 : INFO : collecting all words and their counts
2020-05-03 13:24:15,429 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-05-03 13:24:17,834 : INFO : collected 56887 word types and 2964 unique tags from a corpus of 2964 examples and 4736500 words
2020-05-03 13:24:17,835 : INFO : Loading a fresh vocabulary
2020-05-03 13:24:17,909 : INFO : effective_min_count=2 retains 35097 unique words (61% of original 56887, drops 21790)
2020-05-03 13:24:17,910 : INFO : effective_min_count=2 leaves 4714710 word corpus (99% of original 4736500, drops 21790)
2020-05-03 13:24:18,032 : INFO : deleting the raw counts dictionary of 56887 items
2020-05-03 13:24:18,035 : INFO : sample=0.001 downsamples 35 most-common words
2020-05-03 13:24:18,036 : INFO : downsampling leaves estimated 4433608 word corpus (94.0% of prior 4714710)
2020-05-03 13:24:1

done building vocabulary
start training the model


2020-05-03 13:24:26,681 : INFO : EPOCH 1 - PROGRESS: at 6.24% examples, 214822 words/s, in_qsize 7, out_qsize 0
2020-05-03 13:24:27,768 : INFO : EPOCH 1 - PROGRESS: at 11.88% examples, 223221 words/s, in_qsize 7, out_qsize 0
2020-05-03 13:24:28,769 : INFO : EPOCH 1 - PROGRESS: at 17.81% examples, 228653 words/s, in_qsize 7, out_qsize 0
2020-05-03 13:24:29,787 : INFO : EPOCH 1 - PROGRESS: at 23.14% examples, 229548 words/s, in_qsize 8, out_qsize 0
2020-05-03 13:24:30,794 : INFO : EPOCH 1 - PROGRESS: at 28.81% examples, 231262 words/s, in_qsize 7, out_qsize 0
2020-05-03 13:24:31,831 : INFO : EPOCH 1 - PROGRESS: at 33.74% examples, 230774 words/s, in_qsize 7, out_qsize 0
2020-05-03 13:24:32,850 : INFO : EPOCH 1 - PROGRESS: at 38.73% examples, 230363 words/s, in_qsize 7, out_qsize 0
2020-05-03 13:24:33,851 : INFO : EPOCH 1 - PROGRESS: at 44.70% examples, 231672 words/s, in_qsize 7, out_qsize 0
2020-05-03 13:24:34,864 : INFO : EPOCH 1 - PROGRESS: at 49.87% examples, 230763 words/s, in_qsize

932.8265385627747


2020-05-03 13:39:59,376 : INFO : saved doc2vec_case.model


**Saving the similarity scores of Query with case documents in decreasing order of required format for Doc2Vec model**

In [0]:
f = open('doc2vec-task1.txt','w+')
for z in range(0,40): 
  lst = model.docvecs.most_similar(labels[z+10],topn= 5000)
  rank = 1
  for i in range(len(lst)):
    if lst[i][0][0] == 'C': 
      if lst[i][1] > 0:
        x = 0
        x = lst[i][1]
      f.write("AILA_Q"+str(z+11) +" "+ "Q0" + " "+ lst[i][0][:-4] + " "+  str(rank) +" "+ str(x) +" " + "cat" + "\n")
      rank += 1

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


**uploading goldstd_priorcases.txt file for trec evaluation reference file **

In [0]:
from google.colab import files
uploaded = files.upload()

Saving goldstd_priorcases.txt to goldstd_priorcases.txt


**SCORES Prec@5,10 and MAP**

**Another method**

**Computed the BM25 similarity score**

In [0]:
from gensim.summarization.bm25 import get_bm25_weights
from gensim.summarization.bm25 import BM25


sim = [[] for i in range(40)]

query_no = 0
batch_size = 200 # BM25 on batch of 200 at a time. Insufficient memory in case of all at once.
for t in range((len(data)-50)// batch_size +1): 
  print(t)
  arr = []
  docs_seperation = filtered_docs[:50] + filtered_docs[50+batch_size*t:50+batch_size*(t+1)]
  for elem in docs_seperation:
    arr.append(elem.split())
  corpus = arr
  result = get_bm25_weights(corpus, n_jobs=-1)
  
  for i in range(50,len(result)):
    for z in range(40):
      x = result[z][z]
      sim[z].append(result[z+10][i]/x)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


**Saving the similarity scores of Query with case documents in decreasing order of required format for BM25 method**

In [0]:
f = open("BM25-task1.txt","w+")
for z in range(40):
  lst = []
  assert len(sim[z]) == 2914
  for i in range(len(sim[z])):
    lst.append((sim[z][i],i))
  lst.sort(reverse = True)
  for i in range(len(lst)):
    f.write("AILA_Q"+str(z+11) +" "+ "Q0" + " "+ prior_labels[lst[i][1]][:-4] + " "+  str(i+1) +" "+ str( lst[i][0])+ " " + "cat2" + "\n")
    

**Another Method**

**TF-IDF Method**

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfidf = vect.fit_transform(filtered_docs)
tf_idf = (tfidf * tfidf.T).A

In [0]:
simm = [[] for i in range(40)]
for i in range(50,len(tf_idf)):
  for z in range(40):
    simm[z].append(tf_idf[z+10][i])

**Saving the similarity scores of Query with case documents in decreasing order of required format for TF-IDF model**

In [0]:
f = open("TF-IDF-task1.txt","w+")
for z in range(40):
  lst = []
  assert len(simm[z]) == 2914
  for i in range(len(simm[z])):
    lst.append((simm[z][i],i))
  lst.sort(reverse = True)
  for i in range(len(lst)):
    f.write("AILA_Q"+str(z+11) +" "+ "Q0" + " "+ prior_labels[lst[i][1]][:-4] + " "+  str(i+1) +" "+ str( lst[i][0])+ " " + "cat3" + "\n")
    

**TREC EVALUATION**

In [0]:
!wget https://trec.nist.gov/trec_eval/trec_eval_latest.tar.gz

--2020-05-03 13:56:09--  https://trec.nist.gov/trec_eval/trec_eval_latest.tar.gz
Resolving trec.nist.gov (trec.nist.gov)... 132.163.4.36, 2610:20:6b01:4::36
Connecting to trec.nist.gov (trec.nist.gov)|132.163.4.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 181743 (177K) [application/x-gzip]
Saving to: ‘trec_eval_latest.tar.gz’

trec_eval_latest.ta 100%[===================>] 177.48K   392KB/s    in 0.5s    

2020-05-03 13:56:10 (392 KB/s) - ‘trec_eval_latest.tar.gz’ saved [181743/181743]



In [0]:
!tar -xf trec_eval_latest.tar.gz
!rm -r trec_eval_latest.tar.gz

In [0]:
cd trec_eval-9.0.7/

/content/trec_eval-9.0.7


**SCORES Prec@5,10 and MAP**

In [0]:
!make

gcc -g -I.  -Wall -DVERSIONID=\"9.0.7\"  -o trec_eval trec_eval.c formats.c meas_init.c meas_acc.c meas_avg.c meas_print_single.c meas_print_final.c get_qrels.c get_trec_results.c get_prefs.c get_qrels_prefs.c get_qrels_jg.c form_res_rels.c form_res_rels_jg.c form_prefs_counts.c utility_pool.c get_zscores.c convert_zscores.c measures.c  m_map.c m_P.c m_num_q.c m_num_ret.c m_num_rel.c m_num_rel_ret.c m_gm_map.c m_Rprec.c m_recip_rank.c m_bpref.c m_iprec_at_recall.c m_recall.c m_Rprec_mult.c m_utility.c m_11pt_avg.c m_ndcg.c m_ndcg_cut.c m_Rndcg.c m_ndcg_rel.c m_binG.c m_G.c m_rel_P.c m_success.c m_infap.c m_map_cut.c m_gm_bpref.c m_runid.c m_relstring.c m_set_P.c m_set_recall.c m_set_rel_P.c m_set_map.c m_set_F.c m_num_nonrel_judged_ret.c m_prefs_num_prefs_poss.c m_prefs_num_prefs_ful.c m_prefs_num_prefs_ful_ret.c m_prefs_simp.c m_prefs_pair.c m_prefs_avgjg.c m_prefs_avgjg_Rnonrel.c m_prefs_simp_ret.c m_prefs_pair_ret.c m_prefs_avgjg_ret.c m_prefs_avgjg_Rnonrel_ret.c m_prefs_simp_imp.c 

**FOR DOC2VEC method**

In [0]:
! ./trec_eval -m map -m P.5,10 ../goldstd_priorcases.txt ../doc2vec-task1.txt

map                   	all	0.0612
P_5                   	all	0.0450
P_10                  	all	0.0250


**FOR BM25 method**

In [0]:
! ./trec_eval -m map -m P.5,10 ../goldstd_priorcases.txt ../BM25-task1.txt

map                   	all	0.1506
P_5                   	all	0.0950
P_10                  	all	0.0700


**FOR TF-IDF method**

In [0]:
! ./trec_eval -m map -m P.5,10 ../goldstd_priorcases.txt ../TF-IDF-task1.txt

map                   	all	0.1262
P_5                   	all	0.0800
P_10                  	all	0.0675
